### GenAI_Data_Wrangling
#### Data wrangling of notes so that male notes align with male patients and female notes align with female patients

In [1]:
import os
import pandas as pd

In [2]:
# Directory path for medical_records.csv
directory_path = 'Medical_Records_Dataset/medical_records.csv'

medical_records = pd.read_csv(directory_path)
medical_records.head()

,patient_id,name,date_of_birth,gender,medical_conditions,medications,allergies,last_appointment_date
0,1,Scott Webb,1967-04-28,F,"Mrs, story, security","example, parent, city","each, product, two",2022-07-26
1,2,Rachel Frederick,1977-04-04,M,"performance, participant, surface","be, develop, guy","north, hospital, international",2023-02-14
2,3,Eric Kline,1926-05-18,F,"quite, about, international","station, situation, popular","tree, other, at",2021-04-24
3,4,James Rodriguez,1954-07-20,M,"him, sea, television","about, item, often","discuss, play, somebody",2022-05-26
4,5,David Scott,2015-12-07,M,"make, moment, physical","deep, letter, difference","because, true, blood",2021-05-17


In [3]:
# Directory path for mtsamples.csv
directory_path = 'Medical_Transcriptions/mtsamples.csv'

mtsamples = pd.read_csv(directory_path)
mtsamples.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [4]:
# mtsamples = mtsamples.drop('Unnamed: 0', axis=1)
mtsamples = mtsamples.rename(columns={'Unnamed: 0': 'Patient_Num'})

In [5]:
print(mtsamples.columns)

Index(['Patient_Num', 'description', 'medical_specialty', 'sample_name',
       'transcription', 'keywords'],
      dtype='object')


In [6]:
# Define the set of words searching for that should only apply to Males and the words to exclude that whould only apply to Female
words_include = ['circumcision', 'male', 'man', 'mister','mr.', 'penis', 'penile', 'prostate', 'testes', 'testis', 'testical']
pattern_include = '|'.join(words_include) 
words_exclude = ['female','hysterectomy', 'infant', 'lady', 'miss', 'mrs.', 'madam', 'mz.','pregnant', 'pregnancy', 'vagina', 
                 'vaginal','woman']
pattern_exclude = '|'.join(words_exclude) 

# Initialize a mask with all False values
final_mask = pd.Series([False] * len(mtsamples))

# List of columns for word search 
columns_to_search = ['description', 'transcription', 'keywords']

# Generate a mask for each column and combine them
for column in columns_to_search:
    mask_include = mtsamples[column].str.contains(pattern_include, case=False, na=False)
    mask_exclude = ~mtsamples[column].str.contains(pattern_exclude, case=False, na=False)
    final_mask |= (mask_include & mask_exclude)  # Combine with OR to include matches in any column

# Apply the combined mask to filter the DataFrame
m_filtered_df = mtsamples[final_mask]

# print(m_filtered_df.head(50))

In [7]:
# Male notes
m_filtered_df.count()

Patient_Num          1921
description          1921
medical_specialty    1921
sample_name          1921
transcription        1916
keywords             1520
dtype: int64

In [33]:
# Define the set of words searching for that should only apply to Females and the words to exclude that whould only apply to Males
words_exclude = ['circumcision', 'gentleman', 'infant','male', 'man', 'mister','mr.', 'penis', 'penile', 'prostate', 'testes', 'testis', 'testical','well-child']
pattern_exclude = '|'.join(words_exclude) 
words_include = ['endometrium', 'fimbriae', 'fornix', 'fundus', 'infundibulum', 'myometrium', 'perimetrium','abortion', 'breast','cervical', 'cervix', 'colposcopy', 'female','gynecology','gynecologist','hyperplasia', 'hysteroscopy', 'hysterectomy','lady', 'labia', 'lactate', 'lactation', 'leep', 'mammoplasty', 'mammogram', 'mammography', 'mastectomy', 'miss', 'mrs.', 'madam', 'menses', 'menstrual', 'menopause','miscarriage', 'myomectomy', 'mz.','ovary', 'ovarian' ,'pap smear','period', 'placenta','postmenopausal', 'pregnant', 'pregnancy', 'uterus', 'uterine','vagina', 'vaginal','vulva','woman', 'womb']
pattern_include = '|'.join(words_include) 

# Initialize a mask with all False values
final_mask = pd.Series([False] * len(mtsamples))

# List of columns to search 
columns_to_search = ['description', 'transcription', 'keywords']

# Generate a mask for each column and combine them
for column in columns_to_search:
    mask_include = mtsamples[column].str.contains(pattern_include, case=False, na=False)
    mask_exclude = ~mtsamples[column].str.contains(pattern_exclude, case=False, na=False)
    final_mask |= (mask_include & mask_exclude)  # Combine with OR to include matches in any column

# Apply the combined mask to filter the DataFrame
f_filtered_df = mtsamples[final_mask]

In [34]:
# Female notes
f_filtered_df.count()

Patient_Num          955
description          955
medical_specialty    955
sample_name          955
transcription        945
keywords             798
dtype: int64

In [26]:
print(f_filtered_df.head(50))

     Patient_Num                                        description  \
4              4                                 2-D Echocardiogram   
6              6   Liposuction of the supraumbilical abdomen, re...   
30            30   The patient noted for improving retention of ...   
45            45   SPARC suburethral sling due to stress urinary...   
107          107   Left communicating hydrocele.  Left inguinal ...   
110          110   Pelvic tumor, cystocele, rectocele, and uteri...   
113          113   Left hydrocelectomy.   This is a 67-year-old ...   
122          122   Foul-smelling urine and stomach pain after me...   
126          126                                     Epididymectomy   
172          172   Bladder instillation for chronic interstitial...   
182          182   Visually significant posterior capsule opacit...   
204          204   Laparoscopic-assisted vaginal hysterectomy.  ...   
205          205   Vaginal Hysterectomy.  A weighted speculum wa...   
206   

In [35]:
# Filter 'medical_records' to include only males
male_records = medical_records[medical_records['gender'] == 'M']

# Shuffle the male DataFrame to get a random sample
male_records_shuffled = male_records.sample(frac=1).reset_index(drop=True)

# Reset index on m_filtered_df to ensure sequential indices
m_filtered_df_reset = m_filtered_df.reset_index(drop=True)

In [36]:
male_records_shuffled.count()

patient_id               50208
name                     50208
date_of_birth            50208
gender                   50208
medical_conditions       50208
medications              50208
allergies                50208
last_appointment_date    50208
dtype: int64

In [37]:
m_filtered_df_reset.count()

Patient_Num          1921
description          1921
medical_specialty    1921
sample_name          1921
transcription        1916
keywords             1520
dtype: int64

In [38]:
# Ensure male_records_shuffled has the same number of rows as m_filtered_df_reset
male_records_shuffled = male_records_shuffled.iloc[:len(m_filtered_df_reset)]

In [39]:
male_records_shuffled.count()

patient_id               1921
name                     1921
date_of_birth            1921
gender                   1921
medical_conditions       1921
medications              1921
allergies                1921
last_appointment_date    1921
dtype: int64

In [40]:
# Merge the rows based on their indices
# Since we're aligning by index, join on the indices directly
m_new_df = m_filtered_df_reset.join(male_records_shuffled, lsuffix='_m_filtered', rsuffix='_male_records')

In [41]:
m_new_df.head(10)

,Patient_Num,description,medical_specialty,sample_name,transcription,keywords,patient_id,name,date_of_birth,gender,medical_conditions,medications,allergies,last_appointment_date
0,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart...",70800,Patrick Espinoza,1933-12-16,M,"know, last, past","artist, test, we","lose, believe, music",2022-03-11
1,10,Morbid obesity. Laparoscopic Roux-en-Y gastr...,Bariatrics,Laparoscopic Gastric Bypass - 1,"PREOPERATIVE DIAGNOSIS: , Morbid obesity. ,POS...","bariatrics, morbid obesity, roux-en-y, gastric...",40271,Yolanda Simpson,1950-01-02,M,"property, key, window","cup, artist, tend","forget, organization, year",2022-05-17
2,15,Preoperative visit for weight management with...,Bariatrics,Laparoscopic Gastric Banding - Preop Visit,"HISTORY OF PRESENT ILLNESS: ,I have seen ABC t...","bariatrics, laparoscopic gastric banding, pulm...",97297,Andrew Aguilar,1956-07-01,M,"single, prove, to","system, if, again","fall, local, care",2022-06-07
3,16,Neck exploration; tracheostomy; urgent flexib...,Cardiovascular / Pulmonary,Tracheostomy,"PREOPERATIVE DIAGNOSES,Airway obstruction seco...","cardiovascular / pulmonary, airway, laryngolog...",67395,Joseph Brown,1970-07-12,M,"enough, audience, police","result, through, claim","child, across, ball",2021-08-27
4,18,Fertile male with completed family. Elective...,Urology,Vasectomy - 4,"PROCEDURE: , Elective male sterilization via b...","urology, sterilization, vas, fertile male, bil...",68982,Jennifer Moore,1925-10-27,M,"green, sport, seven","full, voice, challenge","everything, floor, finally",2022-02-05
5,20,Whole body radionuclide bone scan due to pros...,Urology,Whole Body Radionuclide Bone Scan,"INDICATION:, Prostate Cancer.,TECHNIQUE:, 3....","urology, prostate cancer, technetium, whole bo...",2311,Peter Burton,1991-11-01,M,"son, cold, four","nothing, structure, price","evidence, outside, receive",2022-05-02
6,21,Patient discharged after laparoscopic Roux-en...,Bariatrics,Gastric Bypass Summary,"FINAL DIAGNOSES,1. Morbid obesity, status pos...","bariatrics, medifast, laparoscopic roux-en-y g...",12453,James Kelly,1982-11-18,M,"amount, wear, clearly","meeting, material, law","pretty, team, style",2021-06-10
7,22,Normal vasectomy,Urology,Vasectomy - 1,"DESCRIPTION:, The patient was placed in the s...","urology, vasectomy, allis clamp, catgut, hemoc...",40949,Mary Myers,1976-01-28,M,"moment, word, factor","have, account, government","study, significant, meet",2022-11-15
8,23,Voluntary sterility. Bilateral vasectomy. T...,Urology,Vasectomy,"PREOPERATIVE DIAGNOSIS: , Voluntary sterility....","urology, hemiscrotum, bilateral vasectomy, vol...",17330,Elizabeth Dunn,1959-08-07,M,"available, option, finish","south, interesting, hospital","lawyer, trip, visit",2022-10-19
9,26,Hispanic male patient was admitted because of...,Urology,Urology Discharge Summary,"PROCEDURES:, Cystourethroscopy and transurethr...","urology, tur, bun, cystourethroscopy, difficul...",15846,Erik Powers,2016-09-29,M,"part, good, that","hard, tough, floor","amount, fill, nor",2021-10-05


In [42]:
# Filter 'medical_records' to include only Females
female_records = medical_records[medical_records['gender'] == 'F']

# Shuffle the Female DataFrame
female_records_shuffled = female_records.sample(frac=1).reset_index(drop=True)

# Reset index on f_filtered_df to ensure sequential indices
f_filtered_df_reset = f_filtered_df.reset_index(drop=True)

In [43]:
female_records_shuffled.count()

patient_id               49792
name                     49792
date_of_birth            49792
gender                   49792
medical_conditions       49792
medications              49792
allergies                49792
last_appointment_date    49792
dtype: int64

In [44]:
f_filtered_df_reset.count()

Patient_Num          955
description          955
medical_specialty    955
sample_name          955
transcription        945
keywords             798
dtype: int64

In [45]:
# Ensure female_records_shuffled has the same number of rows as f_filtered_df_reset
female_records_shuffled = female_records_shuffled.iloc[:len(f_filtered_df_reset)]

In [46]:
# Merge the rows based on their indices
# Since we're aligning by index, join on the indices directly
f_new_df = f_filtered_df_reset.join(female_records_shuffled, lsuffix='_f_filtered', rsuffix='_female_records')

In [47]:
f_new_df.columns

Index(['Patient_Num', 'description', 'medical_specialty', 'sample_name',
       'transcription', 'keywords', 'patient_id', 'name', 'date_of_birth',
       'gender', 'medical_conditions', 'medications', 'allergies',
       'last_appointment_date'],
      dtype='object')

In [48]:
m_new_df.columns

Index(['Patient_Num', 'description', 'medical_specialty', 'sample_name',
       'transcription', 'keywords', 'patient_id', 'name', 'date_of_birth',
       'gender', 'medical_conditions', 'medications', 'allergies',
       'last_appointment_date'],
      dtype='object')

In [49]:
# Combine the f_new_df and m_new_df DataFrames
combined_df = pd.concat([f_new_df, m_new_df], ignore_index=True)

In [50]:
combined_df.count()

Patient_Num              2876
description              2876
medical_specialty        2876
sample_name              2876
transcription            2861
keywords                 2318
patient_id               2876
name                     2876
date_of_birth            2876
gender                   2876
medical_conditions       2876
medications              2876
allergies                2876
last_appointment_date    2876
dtype: int64

In [51]:
# Write combined_df to an Excel file
combined_df.to_excel('combined_patient_records.xlsx', index=False)